# Ch5

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

sns.set()
%matplotlib inline

/Users/hiroaki/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/hiroaki/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## カテゴライズ変数のエンコーディング
- one-hot:全ての特徴量の中で、一つだけ1となり、ほかは0
- dummy-coding: one-hotの次元から不要な一次元を取り除く
- effect-coding: TODO

In [4]:
# onehotとdummy-encodeで都市空間から家賃を予測
from sklearn  import linear_model

df_rent=pd.DataFrame(
    {'City': ['SF', 'SF', 'SF', 'NYC', 'NYC', 'NYC', 
        'Seattle', 'Seattle', 'Seattle'],
        'Rent': [3999, 4000, 4001, 3499, 3500, 3501, 2499, 2500, 2501]})

df_rent

,City,Rent
0,SF,3999
1,SF,4000
2,SF,4001
3,NYC,3499
4,NYC,3500
5,NYC,3501
6,Seattle,2499
7,Seattle,2500
8,Seattle,2501


In [8]:
one_hot_city=pd.get_dummies(df_rent,prefix='city')
one_hot_city

,Rent,city_NYC,city_SF,city_Seattle
0,3999,0,1,0
1,4000,0,1,0
2,4001,0,1,0
3,3499,1,0,0
4,3500,1,0,0
5,3501,1,0,0
6,2499,0,0,1
7,2500,0,0,1
8,2501,0,0,1


In [56]:
dummy_city=pd.get_dummies(df_rent,prefix='city',drop_first=True)

dummy_city

,Rent,city_SF,city_Seattle
0,3999,1,0
1,4000,1,0
2,4001,1,0
3,3499,0,0
4,3500,0,0
5,3501,0,0
6,2499,0,1
7,2500,0,1
8,2501,0,1


In [55]:
# 線形モデルで禁じ
model=linear_model.LinearRegression()
model.fit(one_hot_city[['city_NYC','city_SF','city_Seattle']]
                   ,one_hot_city['Rent'])


print(model.coef_+model.intercept_)
print(df_rent.groupby('City').mean())

[3500. 4000. 2500.]
         Rent
City         
NYC      3500
SF       4000
Seattle  2500


- one-hotの場合、それぞれの係数にバイアスを足したものが平均となる

In [61]:
model.fit(dummy_city[['city_SF','city_Seattle']]
                  , dummy_city['Rent'])


print(model.coef_+model.intercept_)
print(df_rent.groupby('City').mean())

[4000. 2500.]
         Rent
City         
NYC      3500
SF       4000
Seattle  2500


- dummy_codingの場合、biasはdropした特徴量の平均となる

## カテゴリ数が非常に大きな場合、どのようにすればよいか
- カテゴリ数が多いと、スパースな行列になる。
- これをいかに密な行列に変換するか

### Feature Hashing
- hashtrick を使うと、無限のカテゴリが存在する概念(単語など)を特定の次元に押し込める

In [124]:
# feature hashingでどれぐらい容量が軽減されるのかを確認する

# yelpのユーザーデータの最初の5000行を読み込む
import json

jsons=[]
with open('./dataset/yelp_review.json') as file:
     for i in range(5000):
            jsons.append(json.loads(file.readline()))
            
yelp_revew_df=pd.DataFrame(jsons)

yelp_revew_df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw
2,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw
3,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw
4,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw


In [143]:
# business_idの数
m=len(yelp_revew_df.business_id.unique())

from sklearn.feature_extraction import FeatureHasher

# 中でCountVectorで文字列をベクトルに変換してからhash trickを使っている
h=FeatureHasher(n_features=m,input_type='string')
f=h.transform(yelp_revew_df.business_id)

print(m)
print(np.shape(f.toarray()))
f_arr=f.toarray()


from sys import getsizeof

print('original data size is',getsizeof(yelp_revew_df.business_id))
print('hashed data size is',getsizeof(f))

4477
(5000, 4477)
original data size is 395104
hashed data size is 56


In [152]:
# Count Vector
from sklearn.feature_extraction.text import CountVectorizer

vect=CountVectorizer()
X=vect.fit_transform(yelp_revew_df.business_id)

print(np.shape(X.toarray()))

anlysis=vect.build_analyzer()
anlysis('hello world')

(5000, 5627)


['hello', 'world']

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Bin Counting
- カテゴリ変数を統計的値に変換する

In [162]:
# ctr予測のデータをbin countingで圧縮する
ctr_df=pd.read_csv('./dataset/ctr/ctr200m_1.csv')

ctr_df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1.0,2.0,15706.0,320.0,50.0,1722.0,0.0,35.0,-1.0,79.0
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1.0,0.0,15704.0,320.0,50.0,1722.0,0.0,35.0,100084.0,79.0
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1.0,0.0,15704.0,320.0,50.0,1722.0,0.0,35.0,100084.0,79.0
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1.0,0.0,15706.0,320.0,50.0,1722.0,0.0,35.0,100084.0,79.0
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1.0,0.0,18993.0,320.0,50.0,2161.0,0.0,35.0,-1.0,157.0


In [163]:
ctr_df.keys()

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [193]:
def click_counting(x,bin_col):
    # bin_colで指定したカテゴリごとにクリックした回数/クリックしてない回数をカウント
    clicks=pd.Series(x[x['click']==1][bin_col].value_counts(),name='clicks')
    no_clicks=pd.Series(x[x['click']==0][bin_col].value_counts(),name='no_clicks')
    
    counts=pd.DataFrame([clicks,no_clicks]).T.fillna(0)
    counts['total_counts']=counts['clicks'].astype('int64')+counts['no_clicks'].astype('int64')
    
    return counts

def bin_counting(counts):
    counts['N+']=counts['clicks']/counts['total_counts']
    counts['N-']=counts['no_clicks']/counts['total_counts']
    counts['logN']=sp.log10(counts['N+']/counts['N-'])
                                         
    return counts

In [197]:
counts=click_counting(ctr_df,'site_id')

bin_counting(counts)

/Users/hiroaki/.pyenv/versions/pycoon/lib/python3.6/site-packages/numpy/lib/scimath.py:310: RuntimeWarning: divide by zero encountered in log10
  return nx.log10(x)


,clicks,no_clicks,total_counts,N+,N-,logN
00255fb4,2.0,6.0,8,0.250000,0.750000,-0.477121
006ab81e,2.0,1.0,3,0.666667,0.333333,0.301030
00f5367d,1.0,1.0,2,0.500000,0.500000,0.000000
00f648b7,3.0,19.0,22,0.136364,0.863636,-0.801632
00f7c6fc,0.0,2.0,2,0.000000,1.000000,-inf
0114bb39,0.0,1.0,1,0.000000,1.000000,-inf
011e5414,9.0,20.0,29,0.310345,0.689655,-0.346787
01251c29,0.0,4.0,4,0.000000,1.000000,-inf
01270328,0.0,3.0,3,0.000000,1.000000,-inf
0129a661,0.0,2.0,2,0.000000,1.000000,-inf
